In [1]:
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from awsglue.transforms import *
from pyspark.sql.functions import * 
from pyspark.sql.types import * 
from pyspark.sql import SparkSession
from awsglue.utils import *
from awsglue.dynamicframe import DynamicFrame
import sys  
from datetime import datetime 
import pandas as pd

In [2]:
sc = SparkContext()
glueContext=GlueContext(sc)
spark=glueContext.spark_session

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [20]:
df = spark.read.csv('data/*.csv',header=True)

In [ ]:
# Add Homicide Case ID 

In [22]:
columns_list = {
    'Count of Victims': col('Count of Victims').cast(IntegerType()),
    'Age Group': col('Age Group').cast(StringType()),
    'Sex': col('Sex').cast(StringType()),
    'Method of Killing': col('Method of Killing').cast(StringType()),
    'Domestic Abuse': col('Domestic Abuse').cast(IntegerType()), 
    'Recorded Date': col('Recorded Date').cast(StringType()), 
    'Homicide Offence Type': col('Homicide Offence Type').cast(StringType()), 
    'Solved Status': col('Solved Status').cast(StringType()), 
    'Borough': col('Borough').cast(StringType()), 
    'Officer Observed Ethnicity': col('Officer Observed Ethnicity').cast(StringType()), 
    }
df = df.withColumns(columns_list)

In [23]:
rename_cols = {
    'Count of Victims':'victim_count',
    'Age Group': 'age_group',
    'Sex': 'sex',
    'Method of Killing': 'used_method',
    'Domestic Abuse': 'domestic_abuse_flag',
    'Recorded Date': 'crime_recorded_date',
    'Homicide Offence Type':'homicide_offence_type',
    'Solved Status': 'solved_status',
    'Borough': 'crime_recorded_borough',
    'Officer Observed Ethnicity': 'officer_observed_ethnicity',
}
df = df.withColumnsRenamed(rename_cols)

In [24]:
df = df.withColumn('officer_observed_ethnicity', 
              when( col('officer_observed_ethnicity') == 'Not Reported/Not known', 'Unidentified')\
              .when( col('officer_observed_ethnicity') == 'Other.', 'Other')\
              .otherwise(col('officer_observed_ethnicity')))

In [25]:
df = df.withColumn('sex', 
              when( col('sex') == 'Unknown', 'Unidentified Victim')\
              .otherwise(col('sex')))

In [26]:
df = df.withColumn('used_method', 
              when( col('used_method') == 'Not known/Not Recorded', 'Unknown Method')\
              .otherwise(col('used_method')))

In [29]:
df = df.withColumn('crime_recorded_month', split(df.crime_recorded_date, '-').getItem(0))\
       .withColumn('crime_recorded_year', concat(lit('20'), split(df.crime_recorded_date, '-').getItem(1)))

In [30]:
# Remove old and Add new column as DD MM YYYY in crime recorded date formated string

In [31]:
df = df.drop('crime_recorded_date','officer_observed_ethnicity')

In [32]:
df = df.withColumn(
    "age_category",
    when(trim(col("age_group")) == "0 to 12", "Child")
    .when(trim(col("age_group")) == "13 to 19", "Teen")
    .when(trim(col("age_group")) == "20 to 24", "Young Adult")
    .when(trim(col("age_group")) == "25 to 34", "Adult")
    .when(trim(col("age_group")) == "35 to 44", "Adult")
    .when(trim(col("age_group")) == "45 to 54", "Middle-aged")
    .when(trim(col("age_group")) == "55 to 64", "Pre-senior")
    .when(trim(col("age_group")) == "65 and over", "Senior")
)

In [34]:
df = df.withColumn('is_solved_flag',when(col('solved_status')=='Solved', 1).otherwise(0))

In [ ]:
columns_ordered = ['']
df.select(columns_ordered).show()

In [33]:
display(df)

,victim_count,age_group,sex,used_method,domestic_abuse_flag,homicide_offence_type,solved_status,crime_recorded_borough,officer_observed_ethnicity,crime_recorded_month,crime_recorded_year,age_category
0,1,45 to 54,Male Victim,Knife or Sharp Implement,0,Murder,Solved,Wandsworth,White,Jan,2003,Middle-aged
1,1,25 to 34,Male Victim,Unknown Method,0,Murder,Solved,Wandsworth,Asian,Jun,2003,Adult
2,1,45 to 54,Male Victim,"Physical Assault, no weapon",0,Murder,Solved,Greenwich,Asian,Sep,2003,Middle-aged
3,1,45 to 54,Male Victim,Blunt Implement,0,Murder,Solved,Kingston Upon Thames,White,Sep,2003,Middle-aged
4,1,0 to 12,Female Victim,"Physical Assault, no weapon",0,Murder,Solved,Hounslow,White,Jun,2003,Child
5,1,45 to 54,Male Victim,Knife or Sharp Implement,0,Murder,Solved,Hounslow,Asian,Oct,2003,Middle-aged
6,1,0 to 12,Female Victim,"Physical Assault, no weapon",0,Murder,Solved,Richmond Upon Thames,White,Jan,2003,Child
7,1,13 to 19,Female Victim,Unknown Method,0,Murder,Solved,Richmond Upon Thames,White,Feb,2003,Teen
8,1,25 to 34,Male Victim,Knife or Sharp Implement,0,Murder,Solved,Richmond Upon Thames,Other,Aug,2003,Adult
9,1,20 to 24,Male Victim,Knife or Sharp Implement,0,Murder,Solved,Hillingdon,Asian,Jan,2003,Young Adult
